### weather大类有clear, partly_cloudy, haze, cloudy四种。本代码将训练集中含有这四个标签的图片分类了出来。
fname，fname1，fname2，fname3是最终结果。这四个是数组类型，里面含有clear, partly_cloudy, haze, cloudy对应图像在训练集中的名字

In [1]:
import sys
import os
import subprocess

from six import string_types

# Make sure you have all of these packages installed, e.g. via pip
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
from skimage import io
from scipy import ndimage
from IPython.display import display
%matplotlib inline

In [2]:
PLANET_KAGGLE_ROOT = os.path.abspath("/Users/sunchenxi/Desktop/lab/kaggle/amazon/")
PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'train-jpg')
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, 'train_v2.csv')
assert os.path.exists(PLANET_KAGGLE_ROOT)
assert os.path.exists(PLANET_KAGGLE_JPEG_DIR)
assert os.path.exists(PLANET_KAGGLE_LABEL_CSV)

In [3]:
#读取train set标签
labels_df = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [4]:
# 创建一个list里面有所有标签
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [5]:
label_list

['haze',
 'primary',
 'agriculture',
 'clear',
 'water',
 'habitation',
 'road',
 'cultivation',
 'slash_burn',
 'cloudy',
 'partly_cloudy',
 'conventional_mine',
 'bare_ground',
 'artisinal_mine',
 'blooming',
 'selective_logging',
 'blow_down']

In [7]:
#进行one－hot编码 数据预处理
#对每一个图像的标签进行标记，有此标签为1
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
labels_df.head()#显示几个看看

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,train_2,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0


In [8]:
#统计每一个单标签的图像个数
labels_df_sum=labels_df[label_list].sum().sort_values()
labels_df_sum

blow_down               98
conventional_mine      100
slash_burn             209
blooming               332
artisinal_mine         339
selective_logging      340
bare_ground            862
cloudy                2089
haze                  2697
habitation            3660
cultivation           4477
partly_cloudy         7261
water                 7411
road                  8071
agriculture          12315
clear                28431
primary              37513
dtype: int64

In [9]:
#获取含有tags标签的n张图片
def sample_images(tags, n=None):
    """Randomly sample n images with the specified tags."""
    condition = True
    if isinstance(tags, string_types): #tag是否是string类型
        raise ValueError("Pass a list of tags, not a single tag.")
    for tag in tags:
        condition = condition & labels_df[tag] == 1
    if n is not None:
        return labels_df[condition].sample(n)
    else:
        return labels_df[condition]

In [10]:
def load_image(filename):
    '''Look through the directory tree to find the image you specified
    (e.g. train_10.tif vs. train_10.jpg)'''
    for dirname in os.listdir(PLANET_KAGGLE_ROOT):
        path = os.path.abspath(os.path.join(PLANET_KAGGLE_ROOT, dirname, filename))
        if os.path.exists(path):
            print('Found image {}'.format(path))
            return io.imread(path)
    print('Load failed: could not find image {}'.format(path))
 
 #将图片在数组中的位置变成名称
def sample_to_fname(sample_df, row_idx, suffix='jpg'):
    '''Given a dataframe of sampled images, get the
    corresponding filename.'''
    fname = sample_df.get_value(sample_df.index[row_idx], 'image_name')
    return '{}.{}'.format(fname, suffix)

In [11]:
def plot_rgbn_histo(r, g, b):
    for slice_, name, color in ((r,'r', 'red'),(g,'g', 'green'),(b,'b', 'blue')):
        plt.hist(slice_.ravel(), bins=100, 
                 range=[0,rgb_image.max()], 
                 label=name, color=color, histtype='step')
    plt.legend()

##### 计算出label＝clear的图像个数

In [12]:
len(labels_df)

40479

In [13]:
n=0
for i in range(len(labels_df)):
    if(labels_df["clear"][i]==1):
        n=n+1

In [14]:
n

28431

#### 获得含有clear的图片的名字

In [15]:
fname=[0]*n
s = sample_images(['clear'], n=n)
for i in range(n):
    fname[i]= sample_to_fname(s, i)#获得了文件名

##### 图片名字是：

In [16]:
fname

['train_39244.jpg',
 'train_4144.jpg',
 'train_17060.jpg',
 'train_32393.jpg',
 'train_3467.jpg',
 'train_30089.jpg',
 'train_4966.jpg',
 'train_21562.jpg',
 'train_20056.jpg',
 'train_19024.jpg',
 'train_15766.jpg',
 'train_23675.jpg',
 'train_9987.jpg',
 'train_17559.jpg',
 'train_1193.jpg',
 'train_16296.jpg',
 'train_9684.jpg',
 'train_7024.jpg',
 'train_37656.jpg',
 'train_12377.jpg',
 'train_17127.jpg',
 'train_35908.jpg',
 'train_29069.jpg',
 'train_23427.jpg',
 'train_4404.jpg',
 'train_7694.jpg',
 'train_14214.jpg',
 'train_22191.jpg',
 'train_818.jpg',
 'train_10799.jpg',
 'train_38535.jpg',
 'train_19522.jpg',
 'train_5267.jpg',
 'train_8904.jpg',
 'train_26890.jpg',
 'train_27132.jpg',
 'train_24363.jpg',
 'train_30639.jpg',
 'train_32089.jpg',
 'train_33843.jpg',
 'train_37751.jpg',
 'train_9688.jpg',
 'train_13675.jpg',
 'train_35083.jpg',
 'train_7882.jpg',
 'train_17089.jpg',
 'train_28654.jpg',
 'train_20121.jpg',
 'train_38448.jpg',
 'train_19822.jpg',
 'train_33990.j

##### 同样的方法获取含有partly_cloudy标签的图片名称

In [17]:
n1=0
for i in range(len(labels_df)):
    if(labels_df["partly_cloudy"][i]==1):
        n1=n1+1

In [18]:
n1

7261

In [19]:
fname1=[0]*n1
s = sample_images(['clear'], n=n1)
for i in range(n1):
    fname1[i]= sample_to_fname(s, i)#获得了文件名

In [20]:
fname1

['train_5333.jpg',
 'train_33741.jpg',
 'train_27914.jpg',
 'train_14697.jpg',
 'train_35994.jpg',
 'train_30448.jpg',
 'train_34849.jpg',
 'train_4182.jpg',
 'train_32114.jpg',
 'train_1537.jpg',
 'train_21520.jpg',
 'train_30248.jpg',
 'train_1238.jpg',
 'train_1033.jpg',
 'train_19786.jpg',
 'train_10634.jpg',
 'train_35337.jpg',
 'train_6523.jpg',
 'train_15025.jpg',
 'train_8605.jpg',
 'train_35756.jpg',
 'train_13707.jpg',
 'train_37857.jpg',
 'train_2138.jpg',
 'train_33426.jpg',
 'train_19807.jpg',
 'train_5274.jpg',
 'train_6472.jpg',
 'train_1341.jpg',
 'train_12081.jpg',
 'train_9765.jpg',
 'train_25320.jpg',
 'train_29075.jpg',
 'train_29200.jpg',
 'train_27968.jpg',
 'train_9882.jpg',
 'train_28279.jpg',
 'train_6228.jpg',
 'train_7956.jpg',
 'train_28300.jpg',
 'train_37848.jpg',
 'train_33464.jpg',
 'train_12803.jpg',
 'train_21820.jpg',
 'train_27843.jpg',
 'train_38989.jpg',
 'train_34913.jpg',
 'train_11990.jpg',
 'train_33471.jpg',
 'train_17234.jpg',
 'train_17609.j

##### 获取含有haze的图片的名称

In [21]:
n2=0
for i in range(len(labels_df)):
    if(labels_df["haze"][i]==1):
        n2=n2+1
fname2=[0]*n1
s = sample_images(['clear'], n=n2)
for i in range(n2):
    fname2[i]= sample_to_fname(s, i)#获得了文件名        

In [22]:
fname

['train_39244.jpg',
 'train_4144.jpg',
 'train_17060.jpg',
 'train_32393.jpg',
 'train_3467.jpg',
 'train_30089.jpg',
 'train_4966.jpg',
 'train_21562.jpg',
 'train_20056.jpg',
 'train_19024.jpg',
 'train_15766.jpg',
 'train_23675.jpg',
 'train_9987.jpg',
 'train_17559.jpg',
 'train_1193.jpg',
 'train_16296.jpg',
 'train_9684.jpg',
 'train_7024.jpg',
 'train_37656.jpg',
 'train_12377.jpg',
 'train_17127.jpg',
 'train_35908.jpg',
 'train_29069.jpg',
 'train_23427.jpg',
 'train_4404.jpg',
 'train_7694.jpg',
 'train_14214.jpg',
 'train_22191.jpg',
 'train_818.jpg',
 'train_10799.jpg',
 'train_38535.jpg',
 'train_19522.jpg',
 'train_5267.jpg',
 'train_8904.jpg',
 'train_26890.jpg',
 'train_27132.jpg',
 'train_24363.jpg',
 'train_30639.jpg',
 'train_32089.jpg',
 'train_33843.jpg',
 'train_37751.jpg',
 'train_9688.jpg',
 'train_13675.jpg',
 'train_35083.jpg',
 'train_7882.jpg',
 'train_17089.jpg',
 'train_28654.jpg',
 'train_20121.jpg',
 'train_38448.jpg',
 'train_19822.jpg',
 'train_33990.j

##### 获取含有cloudy标签的图像名称

In [23]:
n3=0
for i in range(len(labels_df)):
    if(labels_df["cloudy"][i]==1):
        n3=n3+1
fname3=[0]*n1
s = sample_images(['clear'], n=n3)
for i in range(n3):
    fname3[i]= sample_to_fname(s, i)#获得了文件名    

In [24]:
fname3

['train_38977.jpg',
 'train_5719.jpg',
 'train_34966.jpg',
 'train_30698.jpg',
 'train_40265.jpg',
 'train_18155.jpg',
 'train_25478.jpg',
 'train_23179.jpg',
 'train_15068.jpg',
 'train_21357.jpg',
 'train_26402.jpg',
 'train_14985.jpg',
 'train_26652.jpg',
 'train_33019.jpg',
 'train_4297.jpg',
 'train_27308.jpg',
 'train_26284.jpg',
 'train_8486.jpg',
 'train_20967.jpg',
 'train_34529.jpg',
 'train_17784.jpg',
 'train_32348.jpg',
 'train_20507.jpg',
 'train_10196.jpg',
 'train_9723.jpg',
 'train_29068.jpg',
 'train_38011.jpg',
 'train_37958.jpg',
 'train_30113.jpg',
 'train_34654.jpg',
 'train_7481.jpg',
 'train_29098.jpg',
 'train_4652.jpg',
 'train_39184.jpg',
 'train_4683.jpg',
 'train_13060.jpg',
 'train_12475.jpg',
 'train_8535.jpg',
 'train_2650.jpg',
 'train_10299.jpg',
 'train_38653.jpg',
 'train_27524.jpg',
 'train_36326.jpg',
 'train_30962.jpg',
 'train_5476.jpg',
 'train_4412.jpg',
 'train_20800.jpg',
 'train_15398.jpg',
 'train_6706.jpg',
 'train_20798.jpg',
 'train_1030